<a href="https://colab.research.google.com/github/AI-Junction/Personalized-Medicine/blob/master/PersonalizedMedRedCancerTreatVer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer



In [4]:
pwd = os.getcwd()
print(pwd)
dirname = os.path.join(pwd, "AllPersonalizedMedicineData")
print(dirname)


/content
/content/AllPersonalizedMedicineData


In [5]:
!pip install kaggle

    100% |████████████████████████████████| 61kB 2.4MB/s 
    100% |████████████████████████████████| 245kB 6.9MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8b/21/3b/a0076243c6ae12a6215b2da515fe06b539aee7217b406e510e
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify


In [6]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

#filename = "/content/.kaggle/kaggle.json"
#filename = "/.kaggle/kaggle.json"
filename = "kaggle.json"
#os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [8]:
!ls -lha kaggle.json

---x-wx--T 1 root root 65 Dec  3 08:27 kaggle.json


In [38]:
os.path.isdir("/root/.kaggle")

True

In [39]:
dir_kaggle = "/root/.kaggle"
if not os.path.isdir(dir_kaggle):
  !mkdir -p ~/.kaggle  
!cp kaggle.json ~/.kaggle/
os.path.isdir(dir_kaggle)

True

In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
# get all kaggle competitions list
#!kaggle competitions list

In [0]:
# to list all datasets from kaggle
#!kaggle datasets list

In [0]:
# To list data files in a competition
#!kaggle competitions files -c titanic

In [0]:
# how to download a file to content folder in colab
#!kaggle datasets download -d stackoverflow/stack-overflow-2018-developer-survey

In [0]:
# how to unzip a file to destination folder
#!unzip stack-overflow-2018-developer-survey.zip -d stack-overflow-2018-developer-survey

In [0]:
#from subprocess import check_output
#print(check_output(["ls", "/content/stack-overflow-2018-developer-survey"]).decode("utf8"))

In [0]:
#!head ~/.kaggle/datasets/stackoverflow/stack-overflow-2018-developer-survey/survey_results_public.csv
#!head /content/stack-overflow-2018-developer-survey/survey_results_public.csv

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!kaggle competitions files -c msk-redefining-cancer-treatment

name                             size  creationDate         
-------------------------------  ----  -------------------  
training_variants.zip            24KB  2017-06-22 22:42:33  
test_variants.zip                47KB  2017-06-22 22:42:33  
training_text.zip                60MB  2017-06-22 22:42:59  
test_text.zip                    99MB  2017-06-22 22:43:13  
stage2_sample_submission.csv.7z  765B  2017-09-26 00:21:29  
stage2_test_variants.csv.7z       7KB  2017-09-26 00:21:29  
stage2_test_text.csv.7z           9MB  2017-09-26 00:21:30  
stage1_solution_filtered.csv.7z   1KB  2017-09-26 01:49:00  
stage_2_private_solution.csv.7z  592B  2017-10-13 22:04:01  


In [0]:
#os.environ['KAGGLE_CONFIG_DIR'] = "/content/.kaggle"
#KAGGLE_CONFIG_DIR=/content/.kaggle

In [0]:
# to transfer files directly from kaggle competition to google drive. In below code training_variants.zip is transferred
!kaggle competitions download -c msk-redefining-cancer-treatment -f training_variants -p "/content/drive/My Drive/All ML Datasets/AllPersonalizedMedicineData"
!kaggle competitions download -c msk-redefining-cancer-treatment -f test_variants -p "/content/drive/My Drive/All ML Datasets/AllPersonalizedMedicineData"
!kaggle competitions download -c msk-redefining-cancer-treatment -f training_text -p "/content/drive/My Drive/All ML Datasets/AllPersonalizedMedicineData"
!kaggle competitions download -c msk-redefining-cancer-treatment -f test_text -p "/content/drive/My Drive/All ML Datasets/AllPersonalizedMedicineData"

training_variants.zip: Skipping, found more recently modified local copy (use --force to force download)
  0% 0.00/47.5k [00:00<?, ?B/s]
100% 47.5k/47.5k [00:00<00:00, 7.13MB/s]
 79% 48.0M/61.0M [00:00<00:00, 64.7MB/s]
100% 61.0M/61.0M [00:00<00:00, 101MB/s] 
 99% 98.0M/99.0M [00:00<00:00, 105MB/s]
100% 99.0M/99.0M [00:00<00:00, 111MB/s]


In [0]:
!ls "/content/drive/My Drive/Machine Learning/All ML Datasets"

Advertising.csv		     diabetes.csv     SMSSpamCollection.tsv
all_digits_mnist_test.csv    iris.data	      SMSSpamreadme
AllPersonalizedMedicineData  iris.names       titanic_test.csv
cifar-10-python		     mnist_test.csv   titanic_train.csv
cifar-10-python.tar.gz	     mnist_train.csv  titanic.xls


In [0]:
# trial only
# titanic_dir = "/content/drive/My Drive/Machine Learning/All ML Datasets"
# print(os.path.join(titanic_dir, "titanic_train.csv"))


In [0]:
# trial only
# titanic_train_df = pd.read_csv(os.path.join(titanic_dir, "titanic_train.csv"))


In [0]:
# trial only
# titanic_train_df.head()

In [0]:
pwd = os.getcwd()
print(pwd)

/content


In [0]:
print(os.listdir())


['.config', 'c:\\Users\\echpar\\.kaggle\\kaggle.json', '.kaggle', 'kaggle.json', 'stack-overflow-2018-developer-survey.zip', 'adc.json', 'sample_data']


In [0]:
#dir = "/content/drive/My Drive/Machine Learning/All ML Datasets/AllPersonalizedMedicineData"
#!unzip "drive/My Drive/All ML Datasets/AllPersonalizedMedicineData/training_variants.zip"

'''
training_variants.zip            24KB  2017-06-22 22:42:33  
test_variants.zip                47KB  2017-06-22 22:42:33  
training_text.zip                60MB  2017-06-22 22:42:59  
test_text.zip                    99MB  2017-06-22 22:43:13  
'''

!unzip "drive/My Drive/All ML Datasets/AllPersonalizedMedicineData/training_variants.zip" -d "drive/My Drive/All ML Datasets/AllPersonalizedMedicineData"
!unzip "drive/My Drive/All ML Datasets/AllPersonalizedMedicineData/test_variants.zip" -d "drive/My Drive/All ML Datasets/AllPersonalizedMedicineData"
!unzip "drive/My Drive/All ML Datasets/AllPersonalizedMedicineData/training_text.zip" -d "drive/My Drive/All ML Datasets/AllPersonalizedMedicineData"
!unzip "drive/My Drive/All ML Datasets/AllPersonalizedMedicineData/test_text.zip" -d "drive/My Drive/All ML Datasets/AllPersonalizedMedicineData"


Archive:  drive/My Drive/All ML Datasets/AllPersonalizedMedicineData/training_variants.zip
replace drive/My Drive/All ML Datasets/AllPersonalizedMedicineData/training_variants? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  drive/My Drive/All ML Datasets/AllPersonalizedMedicineData/test_variants.zip
  inflating: drive/My Drive/All ML Datasets/AllPersonalizedMedicineData/test_variants  
Archive:  drive/My Drive/All ML Datasets/AllPersonalizedMedicineData/training_text.zip
  inflating: drive/My Drive/All ML Datasets/AllPersonalizedMedicineData/training_text  
Archive:  drive/My Drive/All ML Datasets/AllPersonalizedMedicineData/test_text.zip
  inflating: drive/My Drive/All ML Datasets/AllPersonalizedMedicineData/test_text  


In [0]:
#!cp training_variants "drive/My Drive/All ML Datasets/AllPersonalizedMedicineData"

In [0]:
import os.path
os.path.isfile("drive/My Drive/All ML Datasets/AllPersonalizedMedicineData/training_variants.zip") 

True

In [0]:
os.path.isdir(dir)

True

In [0]:
print(os.path.exists("drive/My Drive/Machine Learning/All ML Datasets/AllPersonalizedMedicineData/training_variants"))

True


In [0]:
!ls "/content/drive/My Drive/All ML Datasets/AllPersonalizedMedicineData"

test_text      test_variants	  training_text      training_variants
test_text.zip  test_variants.zip  training_text.zip  training_variants.zip


In [0]:
!ls "drive/My Drive/All ML Datasets/AllPersonalizedMedicineData"

training_variants  training_variants.zip


In [0]:
dirname = "drive/My Drive/All ML Datasets/AllPersonalizedMedicineData"
train_variants_df = pd.read_csv(os.path.join(dirname, "training_variants"))

In [0]:
train_variants_df = pd.read_csv(os.path.join(dirname, "training_variants"))
test_variants_df = pd.read_csv(os.path.join(dirname, "test_variants"))
train_text_df = pd.read_csv(os.path.join(dirname, "training_text"), sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])
test_text_df = pd.read_csv(os.path.join(dirname, "test_text"), sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])
print("Train Variant".ljust(15), train_variants_df.shape)
print("Train Text".ljust(15), train_text_df.shape)
print("Test Variant".ljust(15), test_variants_df.shape)
print("Test Text".ljust(15), test_text_df.shape)    


Train Variant   (3321, 4)
Train Text      (3321, 2)
Test Variant    (5668, 3)
Test Text       (5668, 2)


In [0]:
train_variants_df.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [0]:
#export KAGGLE_USERNAME=cpattekar
#export KAGGLE_KEY=af37bff66adaf8e646c1b35bec1ff209

In [0]:
'''
#for f in os.listdir('../input'):
for f in os.listdir(dirname):
    size_bytes = round(os.path.getsize(os.path.join(dirname, f))/ 1000, 2)
    size_name = ["KB", "MB"]
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    print(f.ljust(20) + str(s).ljust(7) + size_name[i])
'''

'\n#for f in os.listdir(\'../input\'):\nfor f in os.listdir(dirname):\n    size_bytes = round(os.path.getsize(os.path.join(dirname, f))/ 1000, 2)\n    size_name = ["KB", "MB"]\n    i = int(math.floor(math.log(size_bytes, 1024)))\n    p = math.pow(1024, i)\n    s = round(size_bytes / p, 2)\n    print(f.ljust(20) + str(s).ljust(7) + size_name[i])\n'

In [0]:
train_variants_df = pd.read_csv(os.path.join(dirname, "training_variants"))
test_variants_df = pd.read_csv(os.path.join(dirname, "test_variants"))
train_text_df = pd.read_csv(os.path.join(dirname, "training_text"), sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])
test_text_df = pd.read_csv(os.path.join(dirname, "test_text"), sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])
print("Train Variant".ljust(15), train_variants_df.shape)
print("Train Text".ljust(15), train_text_df.shape)
print("Test Variant".ljust(15), test_variants_df.shape)
print("Test Text".ljust(15), test_text_df.shape)    


FileNotFoundError: ignored